In [ ]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

In [ ]:
data_file = open('intents2.json').read()
data = json.loads(data_file)
data

In [ ]:
#4 Extracting data_X(features) and data_Y(Target)

words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow  model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X 
# Iterating over all the intents

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern) # tokenize each pattern 
        words.extend(tokens) #and append tokens to words
        data_X.append(pattern) #appending pattern to data_X
        data_y.append(intent["tag"]) ,# appending the associated tag to each pattern 
    
    # adding the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
# 5 Text to Numbers
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [ ]:
#6 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

In [ ]:
#7 Preprocessing the Input

def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability 
  return return_list

def get_response(intents_list, intents_json): 
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [ ]:
model.save("creater.h5")

In [ ]:
# Interacting with the chatbot
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
    message = input("")
    if message == "0":
      break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

In [ ]:
import tkinter as tk
from tkinter import scrolledtext
from tkinter import PhotoImage
from PIL import Image, ImageTk
from IPython.display import display, Markdown, clear_output

# Function to handle user input and display chatbot responses
def send_message():
    user_message = entry_box.get()
    if user_message.strip() != "":
        entry_box.delete(0, tk.END)
        intents = pred_class(user_message, words, classes)
        result = get_response(intents, data)
        chat_history.insert(tk.END, f"You: {user_message}\n")
        chat_history.insert(tk.END, f"ChatBot: {result}\n\n")
        chat_history.see(tk.END)

# Create the main window
root = tk.Tk()
root.title("LightHouseIndia")

# Load and display the logo
logo_image = Image.open("logo.png")  # Replace with your logo path
logo_image = logo_image.resize((100, 100))  # Resize as needed
logo_photo = ImageTk.PhotoImage(logo_image)

logo_label = tk.Label(root, image=logo_photo)
logo_label.pack(pady=10)

# Create a frame for the chat history
frame = tk.Frame(root)
frame.pack(pady=10)

# Create a scrolled text widget to display the chat history
chat_history = scrolledtext.ScrolledText(frame, width=50, height=20, wrap=tk.WORD)
chat_history.pack(expand=True, fill="both")

# Create a frame for the input box and send button
input_frame = tk.Frame(root)
input_frame.pack(pady=10)

# Create an entry box for user input
entry_box = tk.Entry(input_frame, width=40)
entry_box.grid(row=0, column=0, padx=5)

# Create a send button to trigger sending the message
send_button = tk.Button(input_frame, text="Send", command=send_message)
send_button.grid(row=0, column=1, padx=5)

# Function to handle pressing enter in the entry box
def on_enter(event):
    send_message()

# Bind the enter key to the send_message function
root.bind('<Return>', on_enter)

# Start the main tkinter loop
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import scrolledtext
from tkinter import PhotoImage
from PIL import Image, ImageTk
from IPython.display import display, Markdown, clear_output

# Function to handle user input and display chatbot responses
def send_message():
    user_message = entry_box.get()
    if user_message.strip() != "":
        entry_box.delete(0, tk.END)
        intents = pred_class(user_message, words, classes)
        result = get_response(intents, data)
        chat_history.insert(tk.END, f"You: {user_message}\n", 'user')
        chat_history.insert(tk.END, f"ChatBot: {result}\n\n", 'bot')
        chat_history.see(tk.END)

# Create the main window
root = tk.Tk()
root.title("LightHouseIndia")
root.configure(bg='#f0f0f0')  # Set background color

# Load and display the logo
logo_image = Image.open("logo.png")  # Replace with your logo path
logo_image = logo_image.resize((100, 100))  # Resize as needed
logo_photo = ImageTk.PhotoImage(logo_image)

logo_label = tk.Label(root, image=logo_photo, bg='#f0f0f0')
logo_label.pack(pady=10)

# Create a frame for the chat history
frame = tk.Frame(root, bg='#f0f0f0')
frame.pack(pady=10)

# Create a scrolled text widget to display the chat history
chat_history = scrolledtext.ScrolledText(frame, width=50, height=20, wrap=tk.WORD, bg='#e6e6e6', fg='#000000', font=('Arial', 12))
chat_history.pack(expand=True, fill="both")
chat_history.tag_configure('user', foreground='#007BFF', font=('Arial', 12, 'bold'))  # User text style
chat_history.tag_configure('bot', foreground='#FF5733', font=('Arial', 12, 'italic'))  # Bot text style

# Create a frame for the input box and send button
input_frame = tk.Frame(root, bg='#f0f0f0')
input_frame.pack(pady=10)

# Create an entry box for user input
entry_box = tk.Entry(input_frame, width=40, bg='#ffffff', fg='#000000', font=('Arial', 12))
entry_box.grid(row=0, column=0, padx=5)

# Create a send button to trigger sending the message
send_button = tk.Button(input_frame, text="Send", command=send_message, bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
send_button.grid(row=0, column=1, padx=5)

# Function to handle pressing enter in the entry box
def on_enter(event):
    send_message()

# Bind the enter key to the send_message function
root.bind('<Return>', on_enter)

# Start the main tkinter loop
root.mainloop()

In [ ]:
import tkinter as tk
from tkinter import scrolledtext
from PIL import Image, ImageTk
from IPython.display import display, Markdown, clear_output

# Function to handle user input and display chatbot responses
def send_message(user_message):
    if user_message.strip() != "":
        intents = pred_class(user_message, words, classes)
        result = get_response(intents, data)
        chat_history.insert(tk.END, f"You: {user_message}\n", 'user')
        chat_history.insert(tk.END, f"ChatBot: {result}\n\n", 'bot')
        chat_history.see(tk.END)

# Function to handle button clicks
def on_button_click(pattern):
    send_message(pattern)

# Create the main window
root = tk.Tk()
root.title("LightHouseIndia")
root.configure(bg='#f0f0f0')  # Set background color

# Load and display the logo
logo_image = Image.open("logo.png")  # Replace with your logo path
logo_image = logo_image.resize((100, 100))  # Resize as needed
logo_photo = ImageTk.PhotoImage(logo_image)

logo_label = tk.Label(root, image=logo_photo, bg='#f0f0f0')
logo_label.pack(pady=10)

# Create a frame for the chat history
frame = tk.Frame(root, bg='#f0f0f0')
frame.pack(pady=10)

# Create a scrolled text widget to display the chat history
chat_history = scrolledtext.ScrolledText(frame, width=50, height=20, wrap=tk.WORD, bg='#e6e6e6', fg='#000000', font=('Arial', 12))
chat_history.pack(expand=True, fill="both")
chat_history.tag_configure('user', foreground='#007BFF', font=('Arial', 12, 'bold'))  # User text style
chat_history.tag_configure('bot', foreground='#FF5733', font=('Arial', 12, 'italic'))  # Bot text style

# Create a frame for the buttons
button_frame = tk.Frame(root, bg='#f0f0f0')
button_frame.pack(pady=10)

# Add buttons for each pattern in the intents
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        button = tk.Button(button_frame, text=pattern, command=lambda p=pattern: on_button_click(p), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
        button.pack(pady=2)

# Create a frame for the input box and send button
input_frame = tk.Frame(root, bg='#f0f0f0')
input_frame.pack(pady=10)

# Create an entry box for user input
entry_box = tk.Entry(input_frame, width=40, bg='#ffffff', fg='#000000', font=('Arial', 12))
entry_box.grid(row=0, column=0, padx=5)

# Create a send button to trigger sending the message
send_button = tk.Button(input_frame, text="Send", command=lambda: send_message(entry_box.get()), bg='#007BFF', fg='#ffffff', font=('Arial', 12, 'bold'))
send_button.grid(row=0, column=1, padx=5)

# Function to handle pressing enter in the entry box
def on_enter(event):
    send_message(entry_box.get())

# Bind the enter key to the send_message function
root.bind('<Return>', on_enter)

# Start the main tkinter loop
root.mainloop()
